In [1]:
%load_ext autoreload
%autoreload 2
import _init_paths
import os
import sys
import numpy as np
import argparse
import pprint
import pdb
import time
import cv2
import torch
import pickle
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data.sampler import Sampler
import torchvision.transforms as transforms
import torchvision.datasets as dset
from scipy.misc import imread
from roi_data_layer.roidb import combined_roidb
from roi_data_layer.roibatchLoader import roibatchLoader
from model.utils.config import cfg, cfg_from_file, cfg_from_list, get_output_dir
from model.rpn.bbox_transform import clip_boxes
from model.nms.nms_wrapper import nms
from model.rpn.bbox_transform import bbox_transform_inv
from model.utils.net_utils import save_net, load_net, vis_detections
from model.utils.blob import im_list_to_blob
from model.faster_rcnn.vgg16 import vgg16
from model.faster_rcnn.resnet import resnet
from model.utils.net_utils import weights_normal_init, save_net, load_net, \
      adjust_learning_rate, save_checkpoint, clip_gradient
    

In [2]:
def _get_image_blob(im):
    """Converts an image into a network input.
    Arguments:
    im (ndarray): a color image in BGR order
    Returns:
    blob (ndarray): a data blob holding an image pyramid
    im_scale_factors (list): list of image scales (relative to im) used
      in the image pyramid
    """
    im_orig = im.astype(np.float32, copy=True)
    im_orig -= cfg.PIXEL_MEANS

    im_shape = im_orig.shape
    im_size_min = np.min(im_shape[0:2])
    im_size_max = np.max(im_shape[0:2])

    processed_ims = []
    im_scale_factors = []

    for target_size in cfg.TEST.SCALES:
        im_scale = float(target_size) / float(im_size_min)
        # Prevent the biggest axis from being more than MAX_SIZE
        if np.round(im_scale * im_size_max) > cfg.TEST.MAX_SIZE:
            im_scale = float(cfg.TEST.MAX_SIZE) / float(im_size_max)
        im = cv2.resize(im_orig, None, None, fx=im_scale, fy=im_scale,interpolation=cv2.INTER_LINEAR)
        im_scale_factors.append(im_scale)
        processed_ims.append(im)

    # Create a blob to hold the input images
    blob = im_list_to_blob(processed_ims)

    return blob, np.array(im_scale_factors)

In [3]:
cfg_from_file("cfgs/vgg16.yml")
set_cfgs = ['ANCHOR_SCALES', '[4, 8, 16, 32]', 'ANCHOR_RATIOS', '[0.5,1,2]']
cfg_from_list(set_cfgs)
np.random.seed(cfg.RNG_SEED)
holly_classes = np.asarray(['__background__', 'head'])
isCuda = True
cfg.CUDA = isCuda

In [4]:
def make_cuda(isCuda, tensor):
    if isCuda:
        return tensor.cuda()
    return tensor

In [5]:
class sampler(Sampler):
    def __init__(self, train_size, batch_size):
        self.num_data = train_size
        self.num_per_batch = int(train_size / batch_size)
        self.batch_size = batch_size
        self.range = torch.arange(0,batch_size).view(1, batch_size).long()
        self.leftover_flag = False
        if train_size % batch_size:
            self.leftover = torch.arange(self.num_per_batch*batch_size, train_size).long()
            self.leftover_flag = True

    def __iter__(self):
        rand_num = torch.randperm(self.num_per_batch).view(-1,1) * self.batch_size
        self.rand_num = rand_num.expand(self.num_per_batch, self.batch_size) + self.range
        self.rand_num_view = self.rand_num.view(-1)

        if self.leftover_flag:
            self.rand_num_view = torch.cat((self.rand_num_view, self.leftover),0)

        return iter(self.rand_num_view)

    def __len__(self):
        return self.num_data

In [6]:
def getNumpy(tensor):
    return tensor.data.cpu().numpy()
def sortFilters(filters):
    #Sort filters by L1 Norm
    c = filters.reshape(-1, filters.shape[0])
    c = np.linalg.norm(c, ord=1, axis=0)
    d = np.zeros([2, c.shape[0]])
    i = np.argsort(c)
    d[0,:] = i
    d[1,:] = c[i]
    return d
def removeFilters(filters, percent, threshold=None):
    #Remove filters from numpy 
    t = threshold
    if threshold is None:
        t = np.median(filters[1,:]) - np.std(filters[1,:])
    mask = filters[1, :] > t
    numRemoved_1 = mask.sum()
    numRemoved_2 = int(numRemoved_1 * ((100. - percent) / 100.))
    numOfZeros = numRemoved_1 - numRemoved_2
    newFilters = filters[:,numRemoved_2:].copy()
    newFilters[1, 0:numOfZeros] =0
    return newFilters
def removePercent(filters, KeepPercent, PrunePercent):
    #Remove filters from numpy 
    numOfRemove = filters.shape[1] * PrunePercent / 100.
    numRemoved_1 = int(numOfRemove)
    numRemoved_2 = int(numRemoved_1 * ((100. - KeepPercent) / 100.))
    numOfZeros = numRemoved_1 - numRemoved_2
    newFilters = filters[:,numRemoved_2:].copy()
    print("NumOfZeros:")
    print(numOfZeros)
    newFilters[1, 0:numOfZeros] =0
    return newFilters
def sortRecoverFilters(l1Array, filters):
    #Recover the order of the pruned filters
    b = np.argsort(l1Array[0, :])
    c = l1Array[:,b]
    c = c[0,:].astype(int)
    shape = np.asarray(filters.shape)
    shape[0] = c.shape[0]
    newFilters = np.zeros(shape)
    newFilters = filters[c,:,:,:]
    return newFilters
def sortRecoverBatch(bn_tensor, index):
    #Batch pruning 
    index.sort()
    bn_rmean = bn_tensor.running_mean.cpu().numpy()
    bn_tensor.running_mean = torch.from_numpy(bn_rmean[index]).float().cuda()
    bn_rvar = bn_tensor.running_var.cpu().numpy()
    bn_tensor.running_var = torch.from_numpy(bn_rvar[index]).float().cuda()
    bn_weight = bn_tensor.weight.data.cpu().numpy()
    bn_tensor.weight.data = torch.from_numpy(bn_weight[index]).float().cuda()
    bn_bias = bn_tensor.bias.data.cpu().numpy()
    bn_tensor.bias.data = torch.from_numpy(bn_bias[index]).float().cuda()
def pruneConvLayers(tensor, percent = 10, threshold = None):
    #Prune out channels of convolutional layers
    filters = getNumpy(tensor.weight)
    d = sortFilters(filters)
    e = removeFilters(d, percent, threshold)
    f = sortRecoverFilters(e, filters)
    tensor.weight.data = torch.from_numpy(f).float().cuda()
    tensor.out_channels = f.shape[0]
    rindexes = e[0,:].astype(int).copy()
    return tensor, rindexes
def pruneConvPercent(tensor, KeepPercent = 10, PrunePercent=10):
    #Prune out channels of convolutional layers
    filters = getNumpy(tensor.weight)
    d = sortFilters(filters)
    e = removePercent(d, KeepPercent, PrunePercent)
    f = sortRecoverFilters(e, filters)
    tensor.weight.data = torch.from_numpy(f).float().cuda()
    tensor.out_channels = f.shape[0]
    rindexes = e[0,:].astype(int).copy()
    return tensor, rindexes
def pruneConvLowest(tensor, numKeep):
    filters = getNumpy(tensor.weight)
    d = sortFilters(filters)
    e = d[:,0:numKeep]
    f = sortRecoverFilters(e, filters)
    tensor.weight.data = torch.from_numpy(f).float().cuda()
    tensor.out_channels = f.shape[0]
    rindexes = e[0,:].astype(int).copy()
    return tensor, rindexes
def pruneConvWithIndexes(tensor, rindexes):
    #Prune out channels of convolutional layers keeping rindexes
    filters = getNumpy(tensor.weight)
    tensor.weight.data = torch.from_numpy(filters[rindexes]).float().cuda()
    tensor.out_channels = tensor.weight.data.shape[0]
    return tensor, rindexes
def pruneNextLayer(nextLayerTensor, prevOutput, rindexes=None):
    #Prune input channels of everything
    if "BatchNorm" in str(nextLayerTensor):
        nextLayerTensor.num_features = prevOutput
        sortRecoverBatch(nextLayerTensor, rindexes)
    elif "Conv2d" in str(nextLayerTensor):
        if nextLayerTensor.weight.shape[1] == prevOutput:
            return
        nextLayerTensor.in_channels = prevOutput
        nextConvWeight = getNumpy(nextLayerTensor.weight)
        if not rindexes is None:
            c = nextConvWeight[:,rindexes,:,:]
        else:
            c = nextConvWeight
        nextLayerTensor.weight.data = torch.from_numpy(c).float().cuda()
    elif "Linear" in str(nextLayerTensor):
        n = getNumpy(nextLayerTensor.weight)
        fc = n[:,rindexes]
        nextLayerTensor.in_features = fc.shape[1]
        nextLayerTensor.weight.data = torch.from_numpy(fc).float().cuda()
    return nextLayerTensor
def prune_bottleneck(bottlenecks, prevOutput, rindexesX, keeplast=False):
    for i,bottleneck in enumerate(bottlenecks):
        #Conv1 Prune
        currentTensor = pruneNextLayer(bottleneck.conv1, prevOutput, rindexesX) #Conv1 Input channels Prune
        currentTensor, rindexes = pruneConvLayers(bottleneck.conv1) #Conv1 Output channels Prune
        currentTensor           = pruneNextLayer(bottleneck.bn1, currentTensor.out_channels, rindexes) #BN1 prune
        
        #Conv2 Prune
        currentTensor           = pruneNextLayer(bottleneck.conv2, currentTensor.num_features, rindexes) #Conv2 Input channels Prune
        currentTensor, rindexes = pruneConvLayers(bottleneck.conv2) #Conv2 Input channels Prune
        currentTensor           = pruneNextLayer(bottleneck.bn2, currentTensor.out_channels, rindexes) #BN2 prune
        #Shortcut + conv2
        currentTensor           = pruneNextLayer(bottleneck.conv3, currentTensor.num_features, rindexes) #Conv3 input channels
        
        #Prune shortcut first
        if not bottleneck.downsample is None:
            b_neck = bottleneck.downsample
            pruneNextLayer(b_neck[0], prevOutput, rindexesX) #Shortcut Input Channels
            shorcutconv, rindexes = pruneConvLayers(b_neck[0]) #Shortcut output channels
            pruneNextLayer(b_neck[1], b_neck[0].out_channels, rindexes) #bn1 of shorcut
        
            if not keeplast:
                lastconv, rindexes = pruneConvWithIndexes(bottleneck.conv3, rindexes) #Conv3 output channels
            else:
                lastconv = bottleneck.conv3
        else:
            if not keeplast:
                lastconv, rindexes = pruneConvLowest(bottleneck.conv3, len(rindexesX))
            else:
                lastconv = bottleneck.conv3
            
        lastbn = pruneNextLayer(bottleneck.bn3, currentTensor.out_channels, rindexes) #BN3 prune
        num_output = lastconv.out_channels
        rindexesX = rindexes
    return bottleneck, num_output, rindexes
def prune_bottleneck_percent(bottlenecks, rindexesX, percents):
    for i,bottleneck in enumerate(bottlenecks):
        #Conv1 Prune
        currentTensor = pruneNextLayer(bottleneck.conv1, len(rindexesX), rindexesX) #Conv1 Input channels Prune
        currentTensor, rindexes = pruneConvPercent(bottleneck.conv1, percents[0]) #Conv1 Output channels Prune
        currentTensor           = pruneNextLayer(bottleneck.bn1, currentTensor.out_channels, rindexes) #BN1 prune
        
        #Conv2 Prune
        currentTensor           = pruneNextLayer(bottleneck.conv2, currentTensor.num_features, rindexes) #Conv2 Input channels Prune
        currentTensor, rindexes = pruneConvPercent(bottleneck.conv2, percents[1]) #Conv2 Input channels Prune
        currentTensor           = pruneNextLayer(bottleneck.bn2, currentTensor.out_channels, rindexes) #BN2 prune
        #Shortcut + conv2
        currentTensor           = pruneNextLayer(bottleneck.conv3, currentTensor.num_features, rindexes) #Conv3 input channels
        
        #Prune shortcut first
        if not bottleneck.downsample is None:
            b_neck = bottleneck.downsample
            pruneNextLayer(b_neck[0], len(rindexesX), rindexesX) #Shortcut Input Channels
            shorcutconv, rindexes = pruneConvPercent(b_neck[0], percents[2]) #Shortcut output channels
            pruneNextLayer(b_neck[1], b_neck[0].out_channels, rindexes) #bn1 of shorcut
        
            lastconv, rindexes = pruneConvWithIndexes(bottleneck.conv3, rindexes) #Conv3 output channels
        else:
            lastconv, rindexes = pruneConvLowest(bottleneck.conv3, len(rindexesX))
            
        lastbn = pruneNextLayer(bottleneck.bn3, currentTensor.out_channels, rindexes) #BN3 prune
        num_output = lastconv.out_channels
        rindexesX = rindexes
    return bottleneck, num_output, rindexes
def pruneCBias(convTensor, rindexes):
    convT = convTensor
    bias_data = convT.bias.data.cpu().numpy()
    convT.bias.data = torch.from_numpy(bias_data[rindexes]).float().cuda()
    return convT, rindexes

In [7]:
imdb_name = "holly_prune"
imdb, roidb, ratio_list,ratio_index = combined_roidb(imdb_name)

Loaded dataset `HF__prune` for training
Set proposal method: gt
Appending horizontally-flipped training examples...
HF__prune gt roidb loaded from /export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/data/cache/HF__prune_gt_roidb.pkl
done
Preparing training data...
done
before filtering, there are 2000 images...
after filtering, there are 2000 images...


In [89]:
def runModel(model, imdb, roidb, ratio_list, ratio_index, back):
    batch_size = 1
    cfg.TRAIN.USE_FLIPPED = True
    cfg.USE_GPU_NMS = True
    start_epoch = 0
    max_epochs = 1
    lr_decay_step = 5
    lr_decay_gamma = 0.1
    disp_interval = 100
    output_dir = "./pth_train_dir/vgg16/pruned"
    class_agnostic = False
    session = 1
    train_size = len(roidb)
    lr = 0.0001 #cfg.TRAIN.LEARNING_RATE
    
    sampler_batch = sampler(train_size, batch_size)
    dataset_train = roibatchLoader(roidb, ratio_list, ratio_index, batch_size, imdb.num_classes, training=True)
    dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size, sampler=sampler_batch, num_workers=1)
    im_dataT = torch.FloatTensor(1)
    im_infoT = torch.FloatTensor(1)
    num_boxesT = torch.LongTensor(1)
    gt_boxesT = torch.FloatTensor(1)

    # ship to cuda
    im_dataT = make_cuda(isCuda, im_dataT)
    im_infoT = make_cuda(isCuda, im_infoT)
    num_boxesT = make_cuda(isCuda, num_boxesT)
    gt_boxesT = make_cuda(isCuda, gt_boxesT)

      # make variable
    im_dataT = Variable(im_dataT)
    im_infoT = Variable(im_infoT)
    num_boxesT = Variable(num_boxesT)
    gt_boxesT = Variable(gt_boxesT)

    
    params = []
    for key, value in dict(model.named_parameters()).items():
        value.requires_grad = True

    iters_per_epoch = int(train_size / batch_size)

    for epoch in range(start_epoch, max_epochs + 1):
        # setting to train mode
        model.train()
        loss_temp = 0

        if epoch % (lr_decay_step + 1) == 0:
            lr *= lr_decay_gamma

        data_iter = iter(dataloader_train)
        for step in range(iters_per_epoch):
            data = next(data_iter)
            
            im_dataT.data.resize_(data[0].size()).copy_(data[0])
            im_infoT.data.resize_(data[1].size()).copy_(data[1])
            gt_boxesT.data.resize_(data[2].size()).copy_(data[2])
            num_boxesT.data.resize_(data[3].size()).copy_(data[3])

            model.zero_grad()
            rois, cls_prob, bbox_pred, \
            rpn_loss_cls, rpn_loss_box, \
            RCNN_loss_cls, RCNN_loss_bbox, \
            rois_label = model(im_dataT, im_infoT, gt_boxesT, num_boxesT)

            loss = rpn_loss_cls.mean() + rpn_loss_box.mean() \
               + RCNN_loss_cls.mean() + RCNN_loss_bbox.mean()
            loss_temp += loss.data[0]

            # backward
            if back:
                loss.backward()

In [9]:
def evalNetwork(modelFR, out=False):
    modelFR.eval()
    
    cfg.TRAIN.USE_FLIPPED = False
    imdb_name = "holly_test"
    imdb, roidb, ratio_list, ratio_index = combined_roidb(imdb_name, False)
    imdb.competition_mode(on=True)
    
    save_name = 'faster_rcnn_10'
    num_images = len(imdb.image_index)
    all_boxes = [[[] for _ in range(num_images)]
               for _ in range(imdb.num_classes)]

    output_dir = get_output_dir(imdb, save_name)

    dataset = roibatchLoader(roidb, ratio_list, ratio_index, 1, \
                            imdb.num_classes, training=False, normalize = False)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=1,
                                shuffle=False, num_workers=0,
                                pin_memory=True)
    output_dir = "/tmp/outTest"
    vis = False
    thresh = 0.0
    
    data_iter = iter(dataloader)
    num_images = len(imdb.image_index)

    start = time.time()
    max_per_image = 100

    im_dataC = torch.FloatTensor(1)
    im_infoC = torch.FloatTensor(1)
    num_boxesC = torch.LongTensor(1)
    gt_boxesC = torch.FloatTensor(1)

    # ship to cuda
    im_dataC = make_cuda(isCuda, im_dataC)
    im_infoC = make_cuda(isCuda, im_infoC)
    num_boxesC = make_cuda(isCuda, num_boxesC)
    gt_boxesC = make_cuda(isCuda, gt_boxesC)

      # make variable
    im_dataC = Variable(im_dataC, volatile=True)
    im_infoC = Variable(im_infoC, volatile=True)
    num_boxesC = Variable(num_boxesC, volatile=True)
    gt_boxesC = Variable(gt_boxesC, volatile=True)

    _t = {'im_detect': time.time(), 'misc': time.time()}
    det_file = os.path.join(output_dir, 'detections.pkl')

    empty_array = np.transpose(np.array([[],[],[],[],[]]), (1,0))
    for i in range(num_images):

        data = next(data_iter)
        im_dataC.data.resize_(data[0].size()).copy_(data[0])
        im_infoC.data.resize_(data[1].size()).copy_(data[1])
        gt_boxesC.data.resize_(data[2].size()).copy_(data[2])
        num_boxesC.data.resize_(data[3].size()).copy_(data[3])

        det_tic = time.time()
        rois, cls_prob, bbox_pred, \
        rpn_loss_cls, rpn_loss_box, \
        RCNN_loss_cls, RCNN_loss_bbox, \
        rois_label = modelFR(im_dataC, im_infoC, gt_boxesC, num_boxesC)

        scores = cls_prob.data
        boxes = rois.data[:, :, 1:5]

        if cfg.TEST.BBOX_REG:
              # Apply bounding-box regression deltas
            box_deltas = bbox_pred.data
            if cfg.TRAIN.BBOX_NORMALIZE_TARGETS_PRECOMPUTED:
              # Optionally normalize targets by a precomputed mean and stdev
                if False:#args.class_agnostic:
                    box_deltas = box_deltas.view(-1, 4) * torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_STDS).cuda() \
                               + torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_MEANS).cuda()
                    box_deltas = box_deltas.view(1, -1, 4)
                else:
                    box_deltas = box_deltas.view(-1, 4) * torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_STDS).cuda() \
                               + torch.FloatTensor(cfg.TRAIN.BBOX_NORMALIZE_MEANS).cuda()
                    box_deltas = box_deltas.view(1, -1, 4 * len(imdb.classes))

            pred_boxes = bbox_transform_inv(boxes, box_deltas, 1)
            pred_boxes = clip_boxes(pred_boxes, im_infoC.data, 1)
        else:
              # Simply repeat the boxes, once for each class
            pred_boxes = np.tile(boxes, (1, scores.shape[1]))

        pred_boxes /= data[1][0][2]

        scores = scores.squeeze()
        pred_boxes = pred_boxes.squeeze()
        det_toc = time.time()
        detect_time = det_toc - det_tic
        misc_tic = time.time()
        if vis:
            im = cv2.imread(imdb.image_path_at(i))
            im2show = np.copy(im)
        for j in range(1, imdb.num_classes):
            inds = torch.nonzero(scores[:,j]>thresh).view(-1)
              # if there is det
            if inds.numel() > 0:
                cls_scores = scores[:,j][inds]
                _, order = torch.sort(cls_scores, 0, True)
                if False:#args.class_agnostic:
                    cls_boxes = pred_boxes[inds, :]
                else:
                    cls_boxes = pred_boxes[inds][:, j * 4:(j + 1) * 4]

                cls_dets = torch.cat((cls_boxes, cls_scores.unsqueeze(1)), 1)
                # cls_dets = torch.cat((cls_boxes, cls_scores), 1)
                cls_dets = cls_dets[order]
                keep = nms(cls_dets, cfg.TEST.NMS)
                cls_dets = cls_dets[keep.view(-1).long()]
                if vis:
                    im2show = vis_detections(im2show, imdb.classes[j], cls_dets.cpu().numpy(), 0.3)
                all_boxes[j][i] = cls_dets.cpu().numpy()
            else:
                all_boxes[j][i] = empty_array

          # Limit to max_per_image detections *over all classes*
        if max_per_image > 0:
            image_scores = np.hstack([all_boxes[j][i][:, -1]
                                        for j in range(1, imdb.num_classes)])
            if len(image_scores) > max_per_image:
                image_thresh = np.sort(image_scores)[-max_per_image]
                for j in range(1, imdb.num_classes):
                    keep = np.where(all_boxes[j][i][:, -1] >= image_thresh)[0]
                    all_boxes[j][i] = all_boxes[j][i][keep, :]

        misc_toc = time.time()
        nms_time = misc_toc - misc_tic

        sys.stdout.write('im_detect: {:d}/{:d} {:.3f}s {:.3f}s   \r' \
          .format(i + 1, num_images, detect_time, nms_time))
        sys.stdout.flush()

        if vis:
            cv2.imwrite('result.png', im2show)
            pdb.set_trace()

    with open(det_file, 'wb+') as f:
        pickle.dump(all_boxes, f, pickle.HIGHEST_PROTOCOL)

    print('Evaluating detections')
    ap = imdb.evaluate_detections(all_boxes, output_dir, out)
    return ap

In [90]:
load_model_path = "pth_train_dir/vgg16/finished_models/vgg16_test_2_60000.pth"
fasterRCNN = vgg16(holly_classes, pretrained=False, class_agnostic=False)
fasterRCNN.create_architecture()
checkpoint = torch.load(load_model_path)
fasterRCNN.load_state_dict(checkpoint['model'])
if 'pooling_mode' in checkpoint.keys():
    cfg.POOLING_MODE = checkpoint['pooling_mode']

In [91]:
lFM = {}
filter_ranks = {}

for i, e in enumerate(fasterRCNN.RCNN_base):
    if "Conv" in str(e):
        lFM[e] = None
        filter_ranks[e] = None

def forward(self, input, output):
    lFM[self] = output
    
def backward(self, grad_in, grad_out):
    
    val = lFM[self] * grad_out[0]
    values = torch.sum(val, dim = 0, keepdim = True)
    values = values.sum(dim=2, keepdim = True)
    values = values.sum(dim=3, keepdim = True)[0, :, 0, 0].data
    values = values / (lFM[self].size(0) * lFM[self].size(2) * lFM[self].size(3))
    if filter_ranks[self] is None:
        filter_ranks[self] = torch.FloatTensor(lFM[self].size(1)).zero_().cuda()
    
    lFM[self] = None
    filter_ranks[self] += values
    
    

In [92]:
fasterRCNN = fasterRCNN.cuda()

In [93]:
hooklist = []
for i in hooklist:
    i.remove()

for i, e in enumerate(fasterRCNN.RCNN_base):
    if "Conv" in str(e):
        hooklist.append(e.register_forward_hook(forward))
        hooklist.append(e.register_backward_hook(backward))

In [94]:
runModel(fasterRCNN, imdb, roidb, ratio_list, ratio_index, True)

/export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/lib/model/rpn/rpn.py:68: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  rpn_cls_prob_reshape = F.softmax(rpn_cls_score_reshape)
/export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/lib/model/faster_rcnn/faster_rcnn.py:97: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  cls_prob = F.softmax(cls_score)


In [95]:
def normalize_ranks_per_layer(filter_ranks):
    for i in filter_ranks:
        v = torch.abs(filter_ranks[i])
        v = v / np.sqrt(torch.sum(v * v))
        filter_ranks[i] = v.cpu()

In [96]:
normalize_ranks_per_layer(filter_ranks)

In [97]:
def getSortedIndex(convTensor, filter_ranks):
    ranks = filter_ranks[convTensor].numpy()
    ranks = np.asarray(ranks)
    i = np.argsort(ranks)
    return i.astype(int)

In [98]:
allSortedIndexes = {}

for i, e in enumerate(fasterRCNN.RCNN_base):
    if "Conv" in str(e) and i != 28:
        index = getSortedIndex(e, filter_ranks)
        allSortedIndexes[e] = index

In [99]:
for i in hooklist:
    i.remove()

In [100]:
def getFeaturesMap(self, input, output):
    gpfm = F.avg_pool2d(Variable(output.data.cpu()), kernel_size=(output.data.size()[2], output.data.size()[3])).data.numpy().reshape((1,self.out_channels))
    if len(allFM[self]) == 0:
        allFM[self] = gpfm
        print(len(allFM[self]))
    else:
        print(allFM[self].shape)
        allFM[self] = np.concatenate((allFM[self], gpfm), axis=0)

In [101]:
hooklist = []
for i, e in enumerate(fasterRCNN.RCNN_base):
    if "Conv" in str(e):
        hooklist.append(e.register_forward_hook(getFeaturesMap))

In [102]:
allFM = {}

for i, e in enumerate(fasterRCNN.RCNN_base):
    if "Conv" in str(e):
        allFM[e] = []

In [103]:
runModel(fasterRCNN, imdb, roidb, ratio_list, ratio_index, False)

1
1
1
1
1
1
1
1
1
1
1
1
1


/export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/lib/model/rpn/rpn.py:68: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  rpn_cls_prob_reshape = F.softmax(rpn_cls_score_reshape)
/export/livia/home/vision/lethanh/workspace/faster-rcnn.pytorch/lib/model/faster_rcnn/faster_rcnn.py:97: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  cls_prob = F.softmax(cls_score)


(1, 64)
(1, 64)
(1, 128)
(1, 128)
(1, 256)
(1, 256)
(1, 256)
(1, 512)
(1, 512)
(1, 512)
(1, 512)
(1, 512)
(1, 512)
(2, 64)
(2, 64)
(2, 128)
(2, 128)
(2, 256)
(2, 256)
(2, 256)
(2, 512)
(2, 512)
(2, 512)
(2, 512)
(2, 512)
(2, 512)
(3, 64)
(3, 64)
(3, 128)
(3, 128)
(3, 256)
(3, 256)
(3, 256)
(3, 512)
(3, 512)
(3, 512)
(3, 512)
(3, 512)
(3, 512)
(4, 64)
(4, 64)
(4, 128)
(4, 128)
(4, 256)
(4, 256)
(4, 256)
(4, 512)
(4, 512)
(4, 512)
(4, 512)
(4, 512)
(4, 512)
(5, 64)
(5, 64)
(5, 128)
(5, 128)
(5, 256)
(5, 256)
(5, 256)
(5, 512)
(5, 512)
(5, 512)
(5, 512)
(5, 512)
(5, 512)
(6, 64)
(6, 64)
(6, 128)
(6, 128)
(6, 256)
(6, 256)
(6, 256)
(6, 512)
(6, 512)
(6, 512)
(6, 512)
(6, 512)
(6, 512)
(7, 64)
(7, 64)
(7, 128)
(7, 128)
(7, 256)
(7, 256)
(7, 256)
(7, 512)
(7, 512)
(7, 512)
(7, 512)
(7, 512)
(7, 512)
(8, 64)
(8, 64)
(8, 128)
(8, 128)
(8, 256)
(8, 256)
(8, 256)
(8, 512)
(8, 512)
(8, 512)
(8, 512)
(8, 512)
(8, 512)
(9, 64)
(9, 64)
(9, 128)
(9, 128)
(9, 256)
(9, 256)
(9, 256)
(9, 512)
(9, 512)
(

(66, 64)
(66, 128)
(66, 128)
(66, 256)
(66, 256)
(66, 256)
(66, 512)
(66, 512)
(66, 512)
(66, 512)
(66, 512)
(66, 512)
(67, 64)
(67, 64)
(67, 128)
(67, 128)
(67, 256)
(67, 256)
(67, 256)
(67, 512)
(67, 512)
(67, 512)
(67, 512)
(67, 512)
(67, 512)
(68, 64)
(68, 64)
(68, 128)
(68, 128)
(68, 256)
(68, 256)
(68, 256)
(68, 512)
(68, 512)
(68, 512)
(68, 512)
(68, 512)
(68, 512)
(69, 64)
(69, 64)
(69, 128)
(69, 128)
(69, 256)
(69, 256)
(69, 256)
(69, 512)
(69, 512)
(69, 512)
(69, 512)
(69, 512)
(69, 512)
(70, 64)
(70, 64)
(70, 128)
(70, 128)
(70, 256)
(70, 256)
(70, 256)
(70, 512)
(70, 512)
(70, 512)
(70, 512)
(70, 512)
(70, 512)
(71, 64)
(71, 64)
(71, 128)
(71, 128)
(71, 256)
(71, 256)
(71, 256)
(71, 512)
(71, 512)
(71, 512)
(71, 512)
(71, 512)
(71, 512)
(72, 64)
(72, 64)
(72, 128)
(72, 128)
(72, 256)
(72, 256)
(72, 256)
(72, 512)
(72, 512)
(72, 512)
(72, 512)
(72, 512)
(72, 512)
(73, 64)
(73, 64)
(73, 128)
(73, 128)
(73, 256)
(73, 256)
(73, 256)
(73, 512)
(73, 512)
(73, 512)
(73, 512)
(73, 

(128, 64)
(128, 64)
(128, 128)
(128, 128)
(128, 256)
(128, 256)
(128, 256)
(128, 512)
(128, 512)
(128, 512)
(128, 512)
(128, 512)
(128, 512)
(129, 64)
(129, 64)
(129, 128)
(129, 128)
(129, 256)
(129, 256)
(129, 256)
(129, 512)
(129, 512)
(129, 512)
(129, 512)
(129, 512)
(129, 512)
(130, 64)
(130, 64)
(130, 128)
(130, 128)
(130, 256)
(130, 256)
(130, 256)
(130, 512)
(130, 512)
(130, 512)
(130, 512)
(130, 512)
(130, 512)
(131, 64)
(131, 64)
(131, 128)
(131, 128)
(131, 256)
(131, 256)
(131, 256)
(131, 512)
(131, 512)
(131, 512)
(131, 512)
(131, 512)
(131, 512)
(132, 64)
(132, 64)
(132, 128)
(132, 128)
(132, 256)
(132, 256)
(132, 256)
(132, 512)
(132, 512)
(132, 512)
(132, 512)
(132, 512)
(132, 512)
(133, 64)
(133, 64)
(133, 128)
(133, 128)
(133, 256)
(133, 256)
(133, 256)
(133, 512)
(133, 512)
(133, 512)
(133, 512)
(133, 512)
(133, 512)
(134, 64)
(134, 64)
(134, 128)
(134, 128)
(134, 256)
(134, 256)
(134, 256)
(134, 512)
(134, 512)
(134, 512)
(134, 512)
(134, 512)
(134, 512)
(135, 64)
(13

...

(3735, 64)
(3735, 64)
(3735, 128)
(3735, 128)
(3735, 256)
(3735, 256)
(3735, 256)
(3735, 512)
(3735, 512)
(3735, 512)
(3735, 512)
(3735, 512)
(3735, 512)
(3736, 64)
(3736, 64)
(3736, 128)
(3736, 128)
(3736, 256)
(3736, 256)
(3736, 256)
(3736, 512)
(3736, 512)
(3736, 512)
(3736, 512)
(3736, 512)
(3736, 512)
(3737, 64)
(3737, 64)
(3737, 128)
(3737, 128)
(3737, 256)
(3737, 256)
(3737, 256)
(3737, 512)
(3737, 512)
(3737, 512)
(3737, 512)
(3737, 512)
(3737, 512)
(3738, 64)
(3738, 64)
(3738, 128)
(3738, 128)
(3738, 256)
(3738, 256)
(3738, 256)
(3738, 512)
(3738, 512)
(3738, 512)
(3738, 512)
(3738, 512)
(3738, 512)
(3739, 64)
(3739, 64)
(3739, 128)
(3739, 128)
(3739, 256)
(3739, 256)
(3739, 256)
(3739, 512)
(3739, 512)
(3739, 512)
(3739, 512)
(3739, 512)
(3739, 512)
(3740, 64)
(3740, 64)
(3740, 128)
(3740, 128)
(3740, 256)
(3740, 256)
(3740, 256)
(3740, 512)
(3740, 512)
(3740, 512)
(3740, 512)
(3740, 512)
(3740, 512)
(3741, 64)
(3741, 64)
(3741, 128)
(3741, 128)
(3741, 256)
(3741, 256)
(3741,

(3788, 512)
(3789, 64)
(3789, 64)
(3789, 128)
(3789, 128)
(3789, 256)
(3789, 256)
(3789, 256)
(3789, 512)
(3789, 512)
(3789, 512)
(3789, 512)
(3789, 512)
(3789, 512)
(3790, 64)
(3790, 64)
(3790, 128)
(3790, 128)
(3790, 256)
(3790, 256)
(3790, 256)
(3790, 512)
(3790, 512)
(3790, 512)
(3790, 512)
(3790, 512)
(3790, 512)
(3791, 64)
(3791, 64)
(3791, 128)
(3791, 128)
(3791, 256)
(3791, 256)
(3791, 256)
(3791, 512)
(3791, 512)
(3791, 512)
(3791, 512)
(3791, 512)
(3791, 512)
(3792, 64)
(3792, 64)
(3792, 128)
(3792, 128)
(3792, 256)
(3792, 256)
(3792, 256)
(3792, 512)
(3792, 512)
(3792, 512)
(3792, 512)
(3792, 512)
(3792, 512)
(3793, 64)
(3793, 64)
(3793, 128)
(3793, 128)
(3793, 256)
(3793, 256)
(3793, 256)
(3793, 512)
(3793, 512)
(3793, 512)
(3793, 512)
(3793, 512)
(3793, 512)
(3794, 64)
(3794, 64)
(3794, 128)
(3794, 128)
(3794, 256)
(3794, 256)
(3794, 256)
(3794, 512)
(3794, 512)
(3794, 512)
(3794, 512)
(3794, 512)
(3794, 512)
(3795, 64)
(3795, 64)
(3795, 128)
(3795, 128)
(3795, 256)
(3795,

(3842, 256)
(3842, 512)
(3842, 512)
(3842, 512)
(3842, 512)
(3842, 512)
(3842, 512)
(3843, 64)
(3843, 64)
(3843, 128)
(3843, 128)
(3843, 256)
(3843, 256)
(3843, 256)
(3843, 512)
(3843, 512)
(3843, 512)
(3843, 512)
(3843, 512)
(3843, 512)
(3844, 64)
(3844, 64)
(3844, 128)
(3844, 128)
(3844, 256)
(3844, 256)
(3844, 256)
(3844, 512)
(3844, 512)
(3844, 512)
(3844, 512)
(3844, 512)
(3844, 512)
(3845, 64)
(3845, 64)
(3845, 128)
(3845, 128)
(3845, 256)
(3845, 256)
(3845, 256)
(3845, 512)
(3845, 512)
(3845, 512)
(3845, 512)
(3845, 512)
(3845, 512)
(3846, 64)
(3846, 64)
(3846, 128)
(3846, 128)
(3846, 256)
(3846, 256)
(3846, 256)
(3846, 512)
(3846, 512)
(3846, 512)
(3846, 512)
(3846, 512)
(3846, 512)
(3847, 64)
(3847, 64)
(3847, 128)
(3847, 128)
(3847, 256)
(3847, 256)
(3847, 256)
(3847, 512)
(3847, 512)
(3847, 512)
(3847, 512)
(3847, 512)
(3847, 512)
(3848, 64)
(3848, 64)
(3848, 128)
(3848, 128)
(3848, 256)
(3848, 256)
(3848, 256)
(3848, 512)
(3848, 512)
(3848, 512)
(3848, 512)
(3848, 512)
(384

(3895, 512)
(3895, 512)
(3895, 512)
(3896, 64)
(3896, 64)
(3896, 128)
(3896, 128)
(3896, 256)
(3896, 256)
(3896, 256)
(3896, 512)
(3896, 512)
(3896, 512)
(3896, 512)
(3896, 512)
(3896, 512)
(3897, 64)
(3897, 64)
(3897, 128)
(3897, 128)
(3897, 256)
(3897, 256)
(3897, 256)
(3897, 512)
(3897, 512)
(3897, 512)
(3897, 512)
(3897, 512)
(3897, 512)
(3898, 64)
(3898, 64)
(3898, 128)
(3898, 128)
(3898, 256)
(3898, 256)
(3898, 256)
(3898, 512)
(3898, 512)
(3898, 512)
(3898, 512)
(3898, 512)
(3898, 512)
(3899, 64)
(3899, 64)
(3899, 128)
(3899, 128)
(3899, 256)
(3899, 256)
(3899, 256)
(3899, 512)
(3899, 512)
(3899, 512)
(3899, 512)
(3899, 512)
(3899, 512)
(3900, 64)
(3900, 64)
(3900, 128)
(3900, 128)
(3900, 256)
(3900, 256)
(3900, 256)
(3900, 512)
(3900, 512)
(3900, 512)
(3900, 512)
(3900, 512)
(3900, 512)
(3901, 64)
(3901, 64)
(3901, 128)
(3901, 128)
(3901, 256)
(3901, 256)
(3901, 256)
(3901, 512)
(3901, 512)
(3901, 512)
(3901, 512)
(3901, 512)
(3901, 512)
(3902, 64)
(3902, 64)
(3902, 128)
(3902,

(3949, 64)
(3949, 64)
(3949, 128)
(3949, 128)
(3949, 256)
(3949, 256)
(3949, 256)
(3949, 512)
(3949, 512)
(3949, 512)
(3949, 512)
(3949, 512)
(3949, 512)
(3950, 64)
(3950, 64)
(3950, 128)
(3950, 128)
(3950, 256)
(3950, 256)
(3950, 256)
(3950, 512)
(3950, 512)
(3950, 512)
(3950, 512)
(3950, 512)
(3950, 512)
(3951, 64)
(3951, 64)
(3951, 128)
(3951, 128)
(3951, 256)
(3951, 256)
(3951, 256)
(3951, 512)
(3951, 512)
(3951, 512)
(3951, 512)
(3951, 512)
(3951, 512)
(3952, 64)
(3952, 64)
(3952, 128)
(3952, 128)
(3952, 256)
(3952, 256)
(3952, 256)
(3952, 512)
(3952, 512)
(3952, 512)
(3952, 512)
(3952, 512)
(3952, 512)
(3953, 64)
(3953, 64)
(3953, 128)
(3953, 128)
(3953, 256)
(3953, 256)
(3953, 256)
(3953, 512)
(3953, 512)
(3953, 512)
(3953, 512)
(3953, 512)
(3953, 512)
(3954, 64)
(3954, 64)
(3954, 128)
(3954, 128)
(3954, 256)
(3954, 256)
(3954, 256)
(3954, 512)
(3954, 512)
(3954, 512)
(3954, 512)
(3954, 512)
(3954, 512)
(3955, 64)
(3955, 64)
(3955, 128)
(3955, 128)
(3955, 256)
(3955, 256)
(3955,

In [104]:
def getSortedIndexH(convTensor, allFM):
    Hi_0 = []
    print(allFM[convTensor].shape[1])
    for i in range(allFM[convTensor].shape[1]):
        hist, bin_edges = np.histogram(allFM[convTensor][i,:], bins=10)
        p = hist / allFM[convTensor].shape[0]
        p = p[np.nonzero(p)]
        Hi_0.append((p * np.log(p)).sum())

    Hi_0 = np.asarray(Hi_0)
    d = np.zeros([2, Hi_0.shape[0]])
    i = np.argsort(Hi_0)
    return i.astype(int)

In [83]:
allFM[fasterRCNN.RCNN_base[0]][1,:]

IndexError: index 1 is out of bounds for axis 0 with size 1

In [105]:
allSortedIndexesH = {}

for i, e in enumerate(fasterRCNN.RCNN_base):
    if "Conv" in str(e) and i != 28:
        index = getSortedIndexH(e, allFM)
        allSortedIndexesH[e] = index

64
64
128
128
256
256
256
512
512
512
512
512


In [110]:
allSortedIndexesH[fasterRCNN.RCNN_base[0]]

array([52, 27, 39, 32, 51,  0, 22,  3, 12, 16, 45, 25, 20, 60, 63, 24, 41,
       11, 53,  1, 59,  7, 26,  9, 46,  5, 56, 40, 29, 48, 31,  2, 21, 19,
       50, 44, 43, 61, 14,  6, 18, 23, 36, 13, 17,  8, 37, 57, 55, 42, 35,
       49, 58, 15, 30, 54, 34, 33, 62,  4, 28, 10, 38, 47])

In [107]:
allSortedIndexes[fasterRCNN.RCNN_base[0]]

array([42, 20,  4, 56, 57, 28, 27, 48, 24, 25, 40,  2, 17, 14, 33, 45, 53,
       38,  6, 55,  3, 44, 41, 18, 58, 51, 11, 50, 12, 21, 23, 54, 60, 30,
       35, 43,  1,  9, 13,  0, 49,  7, 15, 29, 52, 63, 22, 46, 10,  5, 19,
       26, 47, 39, 62, 37, 59, 34, 36,  8, 16, 31, 32, 61])

In [127]:
compressionRate = 70
for i, e in enumerate(fasterRCNN.RCNN_base):
    if "Conv" in str(e) and i != 28:
        i1 = allSortedIndexes[e]
        i2 = allSortedIndexesH[e]
        num2Removes = int(i1.shape[0] * compressionRate / 100.)
        n1 = i1[num2Removes:]
        n2 = i2[num2Removes:]
        k1 = i1[:(i1.shape[0] - num2Removes)]
        k2 = i2[:(i1.shape[0] - num2Removes)]
        #print(np.corrcoef(allSortedIndexesH[e], allSortedIndexes[e])[0][1])
        print(np.corrcoef(k1, k2)[0][1])
        print(np.sum(k1 == k2))

-0.11461891783501153
0
-0.1683326532891517
0
-0.2707805860160254
0
-0.13885083720127275
0
0.08362419658893656
0
-0.12155566875967211
0
0.06492736375902151
0
-0.187738621481473
1
0.09211075126627066
1
-0.04059610921128033
0
-0.090295288623937
0
-0.042546779164061514
0
